# 名大会話コーパスのデータ整理

#### 名大会話コーパスには100名以上の会話者が参加して、総計100時間もの会話が文字化されている。
#### 今回はこのデータを整理し、その一部を自動読唇の学習用データとする。

## 学習用コーパス作成

### 名大会話コーパス

https://mmsrv.ninjal.ac.jp/nucc/

### 文字化ルール(一部)
https://mmsrv.ninjal.ac.jp/nucc/nucc_abst.html

・聞き取れる声はすべて文字化し、聞き取れない部分には＊＊＊を入れる。<br>
・相づちと思われる発話は（　　　）の中に入れる。<br>
・上昇調イントネーションは？で示す。<br>
・笑いは＜笑い＞とする。相手の笑いは（＜笑い＞）とする。<br>
・一定の長さ以上の沈黙は＜間＞で示す。<br>
・漢字の発音が問題になりそうな時は【　】 の中に仮名を入れる。<br>
・アルファベット、数字、記号はすべて全角とする。<br>

### 今回の編集ルール
・（）を消す<br>
・＜＞を消す<br>
・【】を消す<br>
・speakerが不明の場合「X:」は「U000」とした

## 準備

In [0]:
import numpy as np
import pandas as pd
import os
import re
import collections

In [3]:
%cd /content

/content


In [4]:
# https://mmsrv.ninjal.ac.jp/nucc/ 名古屋会話コーパス
!wget https://mmsrv.ninjal.ac.jp/nucc/nucc.zip
!unzip nucc.zip

--2019-07-08 06:05:29--  https://mmsrv.ninjal.ac.jp/nucc/nucc.zip
Resolving mmsrv.ninjal.ac.jp (mmsrv.ninjal.ac.jp)... 202.245.100.91
Connecting to mmsrv.ninjal.ac.jp (mmsrv.ninjal.ac.jp)|202.245.100.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2187379 (2.1M) [application/zip]
Saving to: ‘nucc.zip’

nucc.zip            100%[===================>]   2.09M   649KB/s    in 3.3s    

2019-07-08 06:05:33 (649 KB/s) - ‘nucc.zip’ saved [2187379/2187379]

Archive:  nucc.zip
   creating: nucc/
  inflating: nucc/data001.txt        
  inflating: nucc/data002.txt        
  inflating: nucc/data003.txt        
  inflating: nucc/data004.txt        
  inflating: nucc/data005.txt        
  inflating: nucc/data006.txt        
  inflating: nucc/data007.txt        
  inflating: nucc/data008.txt        
  inflating: nucc/data009.txt        
  inflating: nucc/data010.txt        
  inflating: nucc/data011.txt        
  inflating: nucc/data012.txt        
  inflating: nucc/data0

In [0]:
def clean_talk(talk):
    tmp = talk
    
    #括弧を中身ごと削除
    tmp = re.sub('（.+?）', '', tmp)
    tmp = re.sub('＜.+?＞', '', tmp)
    tmp = re.sub('【.+?】', '', tmp)
    
    #改行などを削除
    tmp = re.sub('\n', '', tmp)
    tmp = re.sub('\u30c3', '', tmp)
    tmp = re.sub('\u3000', '', tmp)
    tmp = re.sub('\uff1a', '', tmp)
    
    #一部読みを追加
    tmp = re.sub('Ｑ＆Ａ', 'キューアンドエー', tmp)
    tmp = re.sub('&', 'アンド', tmp)
    tmp = re.sub('＆', 'アンド', tmp)
    tmp = re.sub('○', 'まる', tmp)
    tmp = re.sub('☓', 'ばつ', tmp)
    
    return tmp

In [0]:
N = 129

all_data = pd.DataFrame()

for i in range(1, 1+N):
    filepath = '/content/nucc/data{0:03}.txt'.format(i)


    talk_list = []
    speaker_list = []

    with open(filepath, encoding='utf-8') as f:
        speaker = ""
        for s_line in f:
            if "＠ＥＮＤ" in s_line:
                break
            elif re.match(r'^＠', s_line):
                pass
            elif re.match(r'^％ｃｏｍ', s_line):
                pass
            elif re.match(r'^Ｘ：', s_line):
                speaker = "U000"
                talk = s_line[2:]
                talk = clean_talk(talk)
                speaker_list.append(speaker)
                talk_list.append(talk)
                
            elif re.match(r'^[A-Z]\d+：', s_line):
                speaker = s_line[:4]
                talk = s_line[5:]
                talk = clean_talk(talk)
                speaker_list.append(speaker)
                talk_list.append(talk)
            elif speaker!="": #まだspeakerが決まってない冒頭のコメントは削除
                talk = s_line
                talk = clean_talk(talk)
                speaker_list.append(speaker)
                talk_list.append(talk)
            else:
                pass

    data = pd.DataFrame({"talk" : talk_list,
                        "speaker" : speaker_list})

    data["sex"] = [s[0] for s in data["speaker"]]
    data["num"] = [s[1:] for s in data["speaker"]]
    data["unknown"] = [1 if "＊" in s else 0 for s in data["talk"]]
    
    all_data = pd.concat([all_data, data])

In [7]:
all_data.shape

(133424, 5)

## データフレーム(all_data)の説明

・talk　　　　会話の内容 <br>
・speaker　　名大コーパス中での会話者コード<br>
・sex　　　　会話者の性別(F：女性、M：男性)<br>
・num　　　　会話者のコード<br>
・unknown　　talk中に伏字「＊＊＊」(不明or個人情報などによる)が含まれているかどうか<br>

In [8]:
all_data.head()

,talk,speaker,sex,num,unknown
0,＊＊＊の町というのはちいちゃくって、城壁がこう町全体をぐるっと回ってて、それが城壁の上を歩い...,F107,F,107,1
1,１時間かからないぐらいだね。,F023,F,023,0
2,４、５０分で。,F023,F,023,0
3,そうそう。,F107,F,107,0
4,ほいでさあ、ずっと歩いていたんだけど、そうすと上から、なんか町の中が見れるじゃん。,F107,F,107,0


## 会話者別に会話データが何件あるかの集計
 (F004さんの会話データが7805件と最多であることがわかる)

In [9]:
c = collections.Counter(all_data["speaker"])
c.most_common(10)

[('F004', 7805),
 ('F098', 6172),
 ('F128', 4417),
 ('F093', 3284),
 ('F057', 3108),
 ('F050', 2782),
 ('F001', 2574),
 ('M034', 2535),
 ('F130', 2251),
 ('F032', 2219)]

In [10]:
#話者を絞る
tmp_data = all_data[all_data["speaker"]=="M034"] #東京出身だったため
print(tmp_data.shape)

#「＊＊＊」が入った文章は除外
tmp_data = tmp_data[tmp_data["unknown"]==0]
print(tmp_data.shape)

#空白除外
tmp_data = tmp_data[tmp_data["talk"]!=""]
print(tmp_data.shape)

(2535, 5)
(2481, 5)
(2390, 5)


In [11]:
tmp_data.head(10)

,talk,speaker,sex,num,unknown
1,結構すごい意訳があるから。,M034,M,034,0
3,変わってるよ、これ。,M034,M,034,0
5,変わってる。,M034,M,034,0
8,うん、味が。,M034,M,034,0
16,うん。,M034,M,034,0
17,アイスティーにするとよくわからん。,M034,M,034,0
19,同じなんじゃないの？,M034,M,034,0
27,シロプ入れるのかなあ。,M034,M,034,0
35,なるほど。,M034,M,034,0
36,わかりやすい。,M034,M,034,0


In [0]:
sentences = []

for talk in tmp_data["talk"]:
    tmp = talk
    if tmp[-1]!="。":
        tmp = tmp + "。" #「。」で終わってなければ「。」を追加
    tmp = re.sub("、", "。", tmp)
    sentence = tmp.split("。")[:-1]
    sentences.extend(sentence)

In [13]:
len(sentences)

4832

In [14]:
sentences[:10]

['結構すごい意訳があるから',
 '変わってるよ',
 'これ',
 '変わってる',
 'うん',
 '味が',
 'うん',
 'アイスティーにするとよくわからん',
 '同じなんじゃないの？',
 'シロプ入れるのかなあ']